In [17]:
from Crypto.Util.number import getPrime

while True:
    p = getPrime(110)
    if p % 3 == 2:
        break

a = 0
b = 1
F = GF(p)
E = EllipticCurve(F,[a,b])
assert E.j_invariant() == 0
order = E.order()
print(factor(order)) # This is to check if order has many small prime factors which can make the curve vulnerable to a another attack


# Embedding degree
k=2
assert (p^k - 1) % order == 0
        

P = E.gens()[0]
Po = P.order()
d = randint(2, p-1)
print(f"Secret key: {d}")
Q = d*P
print(Q)

2 * 3 * 7 * 11 * 17 * 67 * 89 * 257 * 3011 * 22961 * 1021730890567921
Secret key: 623294512930608635219411580333806
(575757913339371006661951906238476 : 811726331804692211570061583143873 : 1)


In [ ]:
### SOLUTION - MOV Attack
# Ensure it can be solve in reasonable amount of time.
F2.<x> = GF(p^k)
E2 = E.change_ring(F2)
P2 = E2(P)
Q2 = E2(Q)

# Find a random point with the right behaviour
# Not all points work, so we loop until it does
while True:
    R = E2.random_point()
    Ro = R.order()
    g = gcd(Ro, Po)
    S = (Ro//g)*R
    So = S.order()
    if Po/So in ZZ and Po == So:
        break

# Generate pairings
alpha = P2.weil_pairing(S,Po)
beta = Q2.weil_pairing(S,Po)

# Solve dlog in GF(p^k) instead of E / GF(p)
print("Solving dlog....")
dd = beta.log(alpha)
print(dd)

In [ ]:
### OPTIONAL: Ensure that it can solve solved normaly 
solved_d = P.discrete_log(Q)
assert solved_d == d

In [11]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib

FLAG = open("flag.txt","rb").read()

def encrypt_flag(flag, d):    
    sha1 = hashlib.sha1()
    sha1.update(str(d).encode('ascii'))
    key = sha1.digest()[:16]
    
    # Encrypt flag
    iv = os.urandom(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(pad(FLAG, 16))
    # Prepare encryption to send
    data = {}
    data['iv'] = iv.hex()
    data['encrypted_flag'] = ciphertext.hex()
    return data

enc = encrypt_flag(FLAG, d)
print(enc)

{'iv': '0461c54e586038d67341b0c3d76aa390', 'encrypted_flag': '0faa07310640728d72082b7c56cd24db8a423da82c611a6a087dfb1a96fc25255e85d04a64d041aa5f2200398c41ad26'}


In [15]:
def decrypt_flag(enc_flag, d, iv): 
    
    sha1 = hashlib.sha1()
    sha1.update(str(d).encode('ascii'))
    key = sha1.digest()[:16]

    iv = bytes.fromhex(iv)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(bytes.fromhex(enc_flag)), 16)
    
print(decrypt_flag(enc["encrypted_flag"], d, enc["iv"]))

b'CCSC{3v1l_M0rTy_m1sS3d_th3_eMb3d1ng_d3gree!!}'


In [19]:
template = """E: Elliptic Curve defined by y^2 = x^3 + 1  (mod {0})

Let points on E be:
P = {1}
Q = {2}

such that:
Q = d*P

iv = {3}
key = sha1(d)
cipher = AES_CBC(key, iv)
enc_flag = cipher.encrypt(flag) = {4}
"""

description = template.format(p, P, Q, enc["iv"], enc["encrypted_flag"])

with open("../public/")

E: Elliptic Curve defined by y^2 = x^3 + 1  (mod 1050234795013139803147819743853193)

Let points on E be:
P = (846356572386101725769820855272178 : 163823928752525630055691797248328 : 1)
Q = (23347653264701478231160395818174 : 1028233739711058925412274949097814 : 1)

such that:
Q = d*P

iv = 0461c54e586038d67341b0c3d76aa390
key = sha1(d)
cipher = AES_CBC(key, iv)
enc_flag = cipher.encrypt(flag) = 0faa07310640728d72082b7c56cd24db8a423da82c611a6a087dfb1a96fc25255e85d04a64d041aa5f2200398c41ad26

